In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/datasets/relevantQA_dataset.csv"

In [ ]:
!cp $dataset_path .

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.optim import AdamW
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import random

In [ ]:
# Load your CSV dataset (replace with your actual dataset path)
df = pd.read_csv("relevantQA_dataset.csv")

In [ ]:
# df['relevant'] = df['relevant'].str.lower()

In [ ]:
df['relevant'].unique()

array(['relevant', 'irrelevant'], dtype=object)

In [ ]:
relevant_encoder = LabelEncoder()
df['label'] = relevant_encoder.fit_transform(df['relevant'])

In [ ]:
relevant_encoder.classes_

array(['irrelevant', 'relevant'], dtype=object)

In [ ]:
relevant_encoder.inverse_transform([0])

array(['irrelevant'], dtype=object)

In [ ]:
# df = df.iloc[:1000]

In [ ]:
df.head()

,question,relevant,label
0,What information is accessible on idiopathic p...,relevant,1
1,BACKGROUND: Although RT-qPCR remains the gold-...,relevant,1
2,Hyperinsulinemia has a negative effect on the ...,irrelevant,0
3,"Hi, Dr. Narasi. Thank you for your time. I suf...",relevant,1
4,Hi I have been taking Yaz however I have not c...,relevant,1


In [ ]:
def load_data_from_csv(df, text_column, label_column):
    texts = df[text_column].tolist()
    labels = df[label_column].tolist()

    return texts, labels

In [ ]:
texts, labels = load_data_from_csv(df, 'question', 'label')

In [ ]:
set(labels)

{0, 1}

In [ ]:
# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True,
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128  # Maximum length of tokens
dataset = TextDataset(texts, labels, tokenizer, max_len)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def create_data_loaders(dataset, batch_size):
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

In [ ]:
batch_size = 200
train_data_loader, val_data_loader, test_data_loader = create_data_loaders(dataset, batch_size)

In [ ]:
def initialize_model(device):
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model = model.to(device)
    return model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = initialize_model(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def format_time(seconds):
    """Converts seconds to a string of hours, minutes, and seconds"""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours)}h {int(minutes)}m {int(seconds)}s"

In [ ]:
import time

model_directory = "./relevantQA_bert"

def train_model(model, train_loader, val_loader, device, n_epochs=3):

    optimizer = AdamW(model.parameters(), lr=2e-5)

    prev_val_accuracy = 0
    for epoch in range(n_epochs):

        start_time = time.time()  # Record the start time of the epoch

        total_acc = 0
        for batch in train_loader:
            model.train()
            batch_input_ids = batch['input_ids'].to(device)
            batch_attention_mask = batch['attention_mask'].to(device)
            batch_labels = batch['labels'].to(device)
            optimizer.zero_grad()
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
            loss = outputs[0]
            loss.backward()
            optimizer.step()

            model.eval()
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs[0]
            preds = torch.argmax(logits, dim=1)
            total_acc += (preds == batch_labels).sum().item()

        model.eval()
        total_eval_accuracy = 0
        for batch in val_loader:
            batch_input_ids = batch['input_ids'].to(device)
            batch_attention_mask = batch['attention_mask'].to(device)
            batch_labels = batch['labels'].to(device)
            with torch.no_grad():
                outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
                logits = outputs[0]
                preds = torch.argmax(logits, dim=1)
                total_eval_accuracy += (preds == batch_labels).sum().item()

        # Calculate the duration of the epoch
        end_time = time.time()
        epoch_duration = end_time - start_time
        formatted_time = format_time(epoch_duration)

        avg_val_accuracy = total_eval_accuracy / len(val_loader.dataset)
        avg_train_acc = total_acc / len(train_loader.dataset)
        print(f"Epoch {epoch+1}, Acc: {avg_train_acc:.2f} | Validation acc: {avg_val_accuracy:.2f} | Duration: {formatted_time}")

        # Save model and tokenizer after we got better val loss.
        delta = 0.0001
        if avg_val_accuracy > prev_val_accuracy:
          print(f"{avg_val_accuracy:.4f} > {prev_val_accuracy:.4f}. Saving the model.")
          model.save_pretrained(model_directory)
          tokenizer.save_pretrained(model_directory)
          prev_val_accuracy = avg_val_accuracy

    return model

In [ ]:
# Train model
n_epochs = 20  # Define the number of epochs
trained_model = train_model(model, train_data_loader, val_data_loader, device, n_epochs)

Epoch 1, Acc: 0.99 | Validation acc: 1.00 | Duration: 0h 8m 17s
0.9964 > 0.0000. Saving the model.
Epoch 2, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 3, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 4, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 17s
0.9972 > 0.9964. Saving the model.
Epoch 5, Acc: 1.00 | Validation acc: 0.99 | Duration: 0h 8m 17s
Epoch 6, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 7, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 8, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 9, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 10, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 11, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 12, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 13, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 14, Acc: 1.00 | Validation acc: 1.00 | Duration: 0h 8m 16s
Epoch 15, Acc: 1.00 | Validat

In [ ]:
save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models/"

In [ ]:
!cp -r $model_directory $save_path

In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()
    predictions, true_labels = [], []
    for batch in test_loader:
        batch_input_ids = batch['input_ids'].to(device)
        batch_attention_mask = batch['attention_mask'].to(device)
        batch_labels = batch['labels'].to(device)
        with torch.no_grad():
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs[0]
            preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.tolist())
        true_labels.extend(batch_labels.tolist())
    return predictions, true_labels

In [ ]:
def generate_classification_report(predictions, true_labels):
    report = classification_report(true_labels, predictions, target_names=relevant_encoder.classes_)
    return report

In [ ]:
# Evaluate model
predictions, true_labels = evaluate_model(trained_model, train_data_loader, device)
report = generate_classification_report(predictions, true_labels)
print(report)

In [ ]:
# Evaluate model
predictions, true_labels = evaluate_model(trained_model, test_data_loader, device)
report = generate_classification_report(predictions, true_labels)
print(report)

              precision    recall  f1-score   support

  irrelevant       1.00      1.00      1.00     33992
    relevant       1.00      1.00      1.00     34028

    accuracy                           1.00     68020
   macro avg       1.00      1.00      1.00     68020
weighted avg       1.00      1.00      1.00     68020



In [ ]:
# Completing the predict_text function

def predict_text(model, text, tokenizer, max_len, device):
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True,
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        prediction = torch.argmax(logits, dim=1).cpu().numpy()[0]

    return prediction

# Now, the complete function is provided, and the entire script is corrected. This should ensure the entire process, from data loading to prediction, works smoothly with a CSV input.

In [ ]:
# Load the model and tokenizer from a specified epoch
device = "cuda" if torch.cuda.is_available() else "cpu"

model_directory = "./relevantQA_bert"  # Adjust the path to match where you've saved the model

load_model = BertForSequenceClassification.from_pretrained(model_directory)
load_model.to(device)
load_tokenizer = BertTokenizer.from_pretrained(model_directory)
load_model = load_model.eval()

In [ ]:
# Example prediction
example_text = "I am coughing everyday."
prediction = predict_text(load_model, example_text, load_tokenizer, max_len, device)
predicted_label = relevant_encoder.inverse_transform([prediction])[0]
print(f"Predicted class for example text: {predicted_label}")

Predicted class for example text: relevant


In [ ]:
# Example prediction
example_text = "I like to eat apple."
prediction = predict_text(load_model, example_text, load_tokenizer, max_len, device)
predicted_label = relevant_encoder.inverse_transform([prediction])[0]
print(f"Predicted class for example text: {predicted_label}")

Predicted class for example text: relevant
